# Bot
This a WIP

In [ ]:
print_queue = []
from time import sleep
def print(*data):
    string = ""
    for s in data:
        string += ' {}'.format(str(s))
    print_queue.append(string)
    for x in range(len(print_queue)):
        __builtin__.print('[{}] '.format(x), print_queue.pop())

In [ ]:
# Binance API key and secret, keep these safe. You should change this at least 1/mo.
api_key = 'Your Key'
api_secret = 'Your Secret'

In [ ]:
class ddict(dict):
    def __missing__(self, key):
        value = self[key] = type(self)()  # retain local pointer to value
        return value  # faster to return than dict lookup

# Indicators
__init__(self, name, function)
- Essentially a fancy method holder.
- Must accept a single Coin objecty.
- Returns the result of the indicator.
- Default return is 'init'.

In [ ]:
class Indicator:
    def __init__(self, name, function):
        self.name = name
        self.function = function

    def calc(self, coin):
        result = self.function(coin)
        coin.indicator_results[self.name] = result
        try:
            return result[-1]
        except Exception as e:
            print(e)
            return []

# Coin
__init__(self, client, symbol, market, indicator_results=None)
- Basically a struct, rarely active on its own.
- Contain all useful information

In [ ]:
class Coin:
    def __init__(self, client, symbol, market, indicator_results=None):
        self.opens = []
        self.highs = []
        self.lows = []
        self.closes = []
        self.volumes = []
        self.free = -1
        self.locked = -1
        self.precision = 2
        self.indicator_results = ddict({})
        if indicator_results is not None:
            for indicator in indicator_results:
                self.indicator_results[indicator.name] = 'init'
        self.symbol = symbol
        self.market = market
        self.symbol_long = '{}{}'.format(symbol, market)
        self.last_order_id = -1
        self.need_update = True

# Bot
__init__(self, coins, client=None, indicators=None, fetch_sleep=60, mode='klines', interval=Client.KLINE_INTERVAL_5MINUTE, interval_s=60*5)
###### Do not change bot's mode for now.
- Updates coins on two loops, one recieves data, the other fetches data.
- Updates indicators on a loop, only updates necessary indicators.
- Manages order information.
- Contains a list of indicators that the coins are run through.
- Creates a new thread for every coin that needs to be updated.
- Multithreaded coin updating (shouldn't need to multithread indicators).

In [ ]:
from threading import Thread
from binance.client import Client
from binance.websockets import BinanceSocketManager
from time import sleep
import threading


class Bot:
    def __init__(self, client, indicator_sleep=5, update_sleep=60, mode='klines', 
                 interval=Client.KLINE_INTERVAL_5MINUTE, verbose=False):
        self.client = client
        self.bm = BinanceSocketManager(self.client)
        self.update_sleep = update_sleep
        self.indicator_sleep = indicator_sleep
        self.mode = mode
        self.interval = interval
        self.verbose = verbose
        self.indicators = {}
        # symbol_long : Coin
        self.coins = {}
        # symbol_long : Thread
        self.threads = {}
        self.init = True
        self.running = False

    def start(self):
        self.running = True
        if len(list(self.coins.keys())) <= 0:
            self.running = False
            return -1
        try:
            multiplex_str = []
            if self.mode == 'klines':
                for coin in self.coins.values():
                    multiplex_str.append('{}@kline_{}'.format(coin.symbol_long.lower(), self.interval))
                    # this is the conn_key
                    self.threads['receiver'] = self.bm.start_multiplex_socket(multiplex_str, self.manage_klines)
            elif self.mode == 'trades':
                for coin in self.coins.values():
                    multiplex_str.append('{}@trade'.format(coin.symbol_long.lower()))
                    self.threads['receiver'] = self.bm.start_multiplex_socket(multiplex_str, self.manage_trades)
            else:
                print('No valid mode set.')
                self.running = False
            self.threads['updater'] = Thread(target=self.update_loop, args=[])
            self.threads['updater'].start()
            while self.init:
                sleep(0.25)
            self.threads['indicator updater'] = Thread(target=self.indicator_loop, args=[])
            self.threads['indicator updater'].start()
            self.threads['receiver manager'] = self.bm.start()  # Start receiver.
        except Exception as e:
            print("[bot] Encountered Error!\n",e)
        
    def screen_coins(self, screen):
        results = {}
        for coin in self.coins.values():
            results[coin.symbol] = screen(coin)
    
    def screen_coin(self, coin_symbol_long, screen):
        return screen(self.coins[coin_symbol_long])
    
    def manage_klines(self, message):
        msg = message['data']
        if msg['e'] == 'kline':
            #if self.verbose:
            #    print('[bot] Managing kline for', msg['s'])
            r_kline = msg['k']
            self.coins[msg['s']].opens[-1] = float(r_kline['o'])
            self.coins[msg['s']].highs[-1] = float(r_kline['h'])
            self.coins[msg['s']].lows[-1] = float(r_kline['l'])
            self.coins[msg['s']].closes[-1] = float(r_kline['c'])
            self.coins[msg['s']].volumes[-1] = float(r_kline['v'])
            self.coins[msg['s']].need_update = True
        if self.init:
            self.init = False

    def manage_trades(self, msg):
        # if the trade on a coin is updated:
        coin.need_update = True

    def add_coin(self, coin):
        self.coins[coin.symbol_long] = coin

    def add_coins(self, coins):
        for coin in coins:
            self.add_coin(coin)
    
    def remove_coin(self, coin_symbol_long):
        try:
            if self.threads[coin_symbol_long].is_alive():
                self.threads[coin_symbol_long].join()
        except Exception as e:
            print(e)
        return self.coins.pop(coin_symbol_long, None)
    
    def add_indicator(self, indicator):
        self.indicators[indicator.name] = indicator

    def add_indicators(self, indicators):
        for indicator in indicators:
            self.add_indicator(indicator)

    def calculate_indicators(self, coin):
        if coin.need_update == True:
            for indicator in self.indicators.values():
                if self.verbose:
                    print('[bot] calculating', indicator.name, 'for', coin.symbol_long)
                indicator.calc(coin)
                coin.need_update = False

    def update_loop(self):
        if self.verbose:
            print('[BOT] The update loop has been intiated.')
        while self.running:
            try:
                self.del_keys = []
                keys = list(self.threads.keys())
                for k in keys:
                    print(self.threads[k])
                    if self.threads[k] is None:
                        shh = self.del_keys.append(k)
                    elif isinstance(self.threads[k], Thread):
                        if self.threads[k].is_alive()==False:
                            try:
                                self.threads[k].start()
                            except RuntimeError as e:
                                if self.verbose:
                                    print('[bot][ERR] Thread', k, 'can only be started once? Adding to del_keys.')
                                    print(e)
                                self.del_keys.append(k)
                if self.verbose:
                    print('[bot] Updating all klines.')
                for coin in self.coins.values():
                    self.update_coin(coin)
                if self.init:
                    self.init = False
                if self.verbose:
                    print('Deleting', self.del_keys)
                for key in self.del_keys:
                    del(self.threads[key])
                sleep(self.update_sleep)
            except Exception as e:
                print("[bot] Encountered Error!\n",e)
    
    def add_thread(self, name, target):
        self.threads[name] = target
        
    def remove_thread(self, name):
        if name in self.threads.keys():
            self.del_keys.append(name)
    
    def update_coin(self, coin):
        if self.verbose:
            print('[bot] updating', coin.symbol_long, 'klines.')
        r_klines = self.client.get_klines(
            symbol=coin.symbol_long, interval=self.interval)
        r_bal = client.get_asset_balance(asset=coin.symbol)
        coin.closes = [float(kline[4]) for kline in r_klines]
        coin.opens = [float(kline[1]) for kline in r_klines]
        coin.highs = [float(kline[2]) for kline in r_klines]
        coin.lows = [float(kline[3]) for kline in r_klines]
        coin.volumes = [float(kline[5]) for kline in r_klines]
        coin.free = r_bal['free']
        coin.locked = r_bal['locked']
        coin.need_update = True
    
    def buy(self, coin, amount, price=None):
        self.update_coin(coin)
        if price is None:
            coin.last_order = self.client.order_market_buy()
        else:
            coin.last_order = self.client.order_limit_buy(symbol=coin.symbol_long, quantity=amount, price=round(coin.precision, price))
    
    def sell(self, coin, amount, price=None):
        self.update_coin(coin)
        if price is None:
            coin.last_order = self.client.order_market_sell(symbol=coin.symbol_long, quantity=amount)
        else:
            coin.last_order = self.client.order_limit_sell(symbol=coin.symbol_long, quantity=amount, price=round(coin.precision, price))
        
    def sell_percent(self, coin, percent, price=None):
        self.update_coin(coin)
        rat = (percent/100)
        if coin.free > 0:
            if price is not None:
                self.sell(coin, (coin.free*rat))
        
    def sell_percent(self, coin, percent, price=None):
        self.update_coin(coin)
        rat = (percent/100)
        if coin.free > 0:
            if price is not None:
                self.sell(coin, (coin.free*rat))
    
    def indicator_loop(self):
        while self.running:
            try:
                if self.verbose:
                    print('[bot] indicator_loop updating indicators on updated coins.')
                for coin in self.coins.values():
                    if coin.need_update:
                        if self.verbose:
                            print('[bot]', coin.symbol_long, 'needs update. Queueing.')
                        self.threads[coin.symbol_long] = Thread(target=self.calculate_indicators, args=[coin])
                        if self.verbose:
                            print('[bot] Starting thread for', coin.symbol_long)
                        self.threads[coin.symbol_long].start()
                sleep(self.indicator_sleep)
            except Exception as e:
                print("[bot] Encountered Error!\n",e)